In [6]:
# for some reason the notebook can't find nltk or spacy, use the .py file
import jsonlines
import random
import spacy
import nltk
from nltk.corpus import wordnet

ModuleNotFoundError: No module named 'spacy'

In [4]:
file_path = 'snli_1.0/snli_1.0_dev.jsonl'

data = []
with jsonlines.open(file_path) as reader:
    for obj in reader:
        data.append(obj)

In [5]:
def create_contrast_set(example):
    tokens = nltk.word_tokenize(example['hypothesis'])
    pos_tags = nltk.pos_tag(tokens)
    
    contrasts = []
    
    for i, (word, pos) in enumerate(pos_tags):
        # targeting verbs, adverbs, and adjectives
        if pos.startswith(('VB', 'RB', 'JJ')):
            synsets = wordnet.synsets(word)
            for syn in synsets:
                for lemma in syn.lemmas():
                    if lemma.name() != word:
                        new_tokens = tokens.copy()
                        new_tokens[i] = lemma.name()
                        contrasts.append({
                            'premise': example['premise'],
                            'hypothesis': ' '.join(new_tokens),
                            'label': example['label']
                        })
    
    return contrasts


In [7]:
nlp = spacy.load('en_core_web_sm')

def create_contrast_example(sentence, print_debug=False):
    doc = nlp(sentence)
    new_sentence = sentence
    modifications = []
    
    content_pos = ['NOUN', 'VERB', 'ADJ', 'ADV']
    
    for token in doc:
        if token.pos_ in content_pos:
            synsets = wordnet.synsets(token.text)
            alternatives = []
            
            for syn in synsets:
                for lemma in syn.lemmas():
                    if lemma.name().lower() != token.text.lower():
                        alternatives.append(lemma.name())
            
            if alternatives:
                replacement = random.choice(alternatives)
                old_word = token.text
                new_sentence = new_sentence.replace(old_word, replacement)
                modifications.append((old_word, replacement))
    
    if print_debug and modifications:
        print("\nOriginal:", sentence)
        print("Modified:", new_sentence)
        print("Changes:", modifications)
        
    return new_sentence, modifications

NameError: name 'spacy' is not defined

In [ ]:
def create_contrast_dataset(data, sample_rate=0.1):
    contrast_set = []
    
    for idx, example in enumerate(data):
        original_premise = example['sentence1']
        original_hypothesis = example['sentence2']
        
        # so modifying premise functionality can be added
        modify_choice = random.choice(['hypothesis'])
        
        new_premise = original_premise
        new_hypothesis = original_hypothesis
        
        should_print = random.random() < sample_rate
        
        # if modify_choice in ['premise', 'both']:
        #     new_premise, premise_mods = create_contrast_example(
        #         original_premise, 
        #         print_debug=should_print
        #     )
            
        if modify_choice in ['hypothesis', 'both']:
            new_hypothesis, hypothesis_mods = create_contrast_example(
                original_hypothesis, 
                print_debug=should_print
            )
        
        contrast_example = {
            'original_premise': original_premise,
            'original_hypothesis': original_hypothesis,
            'contrast_premise': new_premise,
            'contrast_hypothesis': new_hypothesis,
            'gold_label': example['gold_label']
        }
        
        contrast_set.append(contrast_example)
        
    return contrast_set

In [ ]:
# print 10% of new examples
contrast_dataset = create_contrast_dataset(data, sample_rate=0.1)

output_path = 'snli_contrast_set.jsonl'
with jsonlines.open(output_path, mode='w') as writer:
    writer.write_all(contrast_dataset)

# stats
print(f"\nTotal examples processed: {len(data)}")
print(f"Contrast examples created: {len(contrast_dataset)}")
